# Setup

## Download files

**Small-scale (~1M images)**

Go to https://deploy.laion.ai/8f83b608504d46bb81708ec86e912220/embeddings/metadata/ and download JUST `metadata_0.parquet`.

Download the small-scale index from https://drive.google.com/drive/folders/11QRjRhTANy5mLquPBV-gIlAjz-NgqHby?usp=sharing.

Put these two files in the following structure:
```
laion_400m_subset/
  |- metadata/
      |- metadata_0.parquet
  |- image.index
```

The corresponding index definition file is `indices_laion_small.json`

**Large-scale**

Download all parquet files from https://deploy.laion.ai/8f83b608504d46bb81708ec86e912220/embeddings/metadata/.

Next, download an `image.index` file from https://the-eye.eu/public/AI/cah/laion400m-indexes/

We assume the following file structure. The path to the root `data` folder is defined in `indices_paths.json` and can be modified as needed:

```
laion_400m/
  |- metadata/
      |- metadata_0.parquet 
      |- ...
  |- image.index
```


## Install dependencies:

Install CLIP:
```
pip install ftfy regex tqdm
pip install git+https://github.com/openai/CLIP.git
```

Dependending on if you want to GPU-accelerate the nearest-neighbors index:

```pip install faiss-gpu``` OR ```pip install faiss-gpu```

# Using the KNN Index

In [1]:
from clip_knn_service import create

knn_service = create(
    indices_paths="indices_laion_small.json",
    
    # Enable hdf5 caching for the metadata. This reduces metadata memory 
    # use to near zero, but on the first run will take a bit of time to 
    # create the memory-mapped hdf5 files
    enable_hdf5=True,
    
    # Use an index with memory mapping, decreasing memory use to zero.
    enable_faiss_memory_mapping=True
)

/Users/edwardvendrow/miniconda3/envs/misc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


warming up with batch size 1 on cpu


/Users/edwardvendrow/Documents/Research/agile_modeling/google-research/agile_modeling/knn/load_clip.py:105: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/BinaryOps.cpp:607.)
  model.encode_image(image_tensor)


done warming up in 0.35264015197753906s


In [2]:
# Get a result from the knn index

results = knn_service.query(
        text_input="gourmet tuna", # can accept text, and image, an image URL, or an embedding
#         image_input=None,
#         image_url_input=None,
#         embedding_input=query,
        modality="image",
        num_images=100,
        num_result_ids=100,
#         use_mclip=False,
        deduplicate=True,
    )

results[0]

{'NSFW': 'UNLIKELY',
 'caption': 'Tuna in Water',
 'image_path': '000004410',
 'url': 'https://gracefoods.com/components/com_jshopping/files/img_products/thumb_cc-tuna-water.jpg',
 'id': 191470,
 'similarity': 0.3783961534500122,
 'embedding': array([ 2.04808987e-03, -5.39444722e-02,  1.42260240e-02,  1.22351442e-02,
         1.54693713e-02,  9.04668309e-03, -2.65145637e-02,  3.85971479e-02,
         3.88641260e-03,  8.16184748e-03,  5.12022467e-04,  2.30057314e-02,
         5.81855848e-02, -1.54388594e-02,  4.34790030e-02, -4.37078392e-03,
         7.35329092e-02,  1.98020153e-02,  1.50650926e-02, -1.44334073e-04,
        -1.85022220e-01,  2.18768027e-02, -3.31965983e-02,  6.59812940e-03,
        -1.55456504e-02,  4.39976975e-02,  1.72848105e-02,  1.94816440e-02,
        -2.53246110e-02, -2.45999615e-03,  4.04278422e-03,  6.76975679e-03,
         3.20981815e-02,  2.09614560e-02, -1.33945839e-02, -2.82384679e-02,
        -8.50662887e-02, -3.99396569e-02, -1.82001572e-02,  3.69190127e-0

# Metadata provider

Here we use the metadata provider to query information given an image's ID

In [3]:
image_id = results[0]['id']
resouce_name = next(iter(knn_service.clip_resources.keys()))
metadata = knn_service.clip_resources[resouce_name].metadata_provider.get([image_id])

metadata

[{'NSFW': b'UNLIKELY',
  'caption': b'Tuna in Water',
  'image_path': b'000004410',
  'url': b'https://gracefoods.com/components/com_jshopping/files/img_products/thumb_cc-tuna-water.jpg'}]